In [1]:
# Import des bibiliothèques nécessaires:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_score, KFold, LeaveOneOut, StratifiedKFold, GridSearchCV

from sklearn.model_selection import train_test_split

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
from numpy import where

from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import warnings
warnings.filterwarnings('ignore')

In [2]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [3]:
# Définition de la fonction variance:
def variance(data):
     # Number of observations
     n = len(data)
     # Mean of the data
     mean = sum(data) / n
     # Square deviations
     deviations = [(x - mean) ** 2 for x in data]
     # Variance
     variance = sum(deviations) / n
     return variance

### Preprocessing des données: voir google colab

In [4]:
#On charge le dataframe preprocessed:
df= pd.read_csv('df.csv')

In [5]:
df.head()

,sk_id_curr,target,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,...,instal_amt_instalment_mean,instal_amt_instalment_sum,instal_amt_payment_min,instal_amt_payment_max,instal_amt_payment_mean,instal_amt_payment_sum,instal_days_entry_payment_max,instal_days_entry_payment_mean,instal_days_entry_payment_sum,instal_count
0,100002.0,1.0,0.0,0.0,0.0,0.0,202500.0,406597.5,24700.5,351000.0,...,11559.247105,219625.695,9251.775,53093.745,11559.247105,219625.695,-49.0,-315.421053,-5993.0,19.0
1,100003.0,0.0,1.0,0.0,1.0,0.0,270000.0,1293502.5,35698.5,1129500.0,...,64754.586000,1618864.650,6662.970,560835.360,64754.586000,1618864.650,-544.0,-1385.320000,-34633.0,25.0
2,100004.0,0.0,0.0,1.0,0.0,0.0,67500.0,135000.0,6750.0,135000.0,...,7096.155000,21288.465,5357.250,10573.965,7096.155000,21288.465,-727.0,-761.666667,-2285.0,3.0
3,100006.0,0.0,1.0,0.0,0.0,0.0,135000.0,312682.5,29686.5,297000.0,...,62947.088438,1007153.415,2482.920,691786.890,62947.088438,1007153.415,-12.0,-271.625000,-4346.0,16.0
4,100007.0,0.0,0.0,0.0,0.0,0.0,121500.0,513000.0,21865.5,513000.0,...,12666.444545,835985.340,0.180,22678.785,12214.060227,806127.975,-14.0,-1032.242424,-68128.0,66.0


### Séparation des données:

In [6]:
# Split des données en train & test
X= df.drop(['target'],axis=1)
y= df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,shuffle=True, stratify=y, random_state=42)

del df
gc.collect()

0

In [7]:
test_target_count = y_test.value_counts()
print('Class 0:', test_target_count[0])
print('Class 1:', test_target_count[1])
print('Proportion:', round(test_target_count[0] / test_target_count[1], 2), ': 1')

Class 0: 113067
Class 1: 9929
Proportion: 11.39 : 1


In [8]:
train_target_count = y_train.value_counts()
print('Class 0:', train_target_count[0])
print('Class 1:', train_target_count[1])
print('Proportion:', round(train_target_count[0] / train_target_count[1], 2), ': 1')

Class 0: 169598
Class 1: 14894
Proportion: 11.39 : 1


### SMOTE for unbalanced class: training set only

In [9]:
# Oversample and plot imbalanced dataset with SMOTE
# summarize class distribution
print("class distribution before SMOTE:")
counter = Counter(y_train)
print(counter)
print('\n')

# transform the dataset
oversample = SMOTE()
X_train_s, y_train_s= oversample.fit_resample(X_train, y_train)

# summarize the new class distribution
print("class distribution after SMOTE:")
counter = Counter(y_train_s)
print(counter)

class distribution before SMOTE:
Counter({0.0: 169598, 1.0: 14894})


class distribution after SMOTE:
Counter({0.0: 169598, 1.0: 169598})


In [10]:
print(stop)

NameError: name 'stop' is not defined

# \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

### Comparaison des modèles de classification avec cross_val_score:

#### LightGBM classifier:

In [ ]:
# LightGBM classifier:
clf_lgbm = LGBMClassifier()
# Validation croisée avec cross_val_score:
# Metric:  ‘roc_auc’
with timer(" process cross_val_score for lgbm"): # process cross_val_score for lgbm - done in 61s
        scores_lgbm= cross_val_score(clf_lgbm,X_train_s, y_train_s, cv= 4, scoring='roc_auc') 

In [ ]:
print(scores_lgbm) # [0.88162016 0.99997028 0.99998038 0.99996053]
print(scores_lgbm.mean()) # 0.9703828383557017
print(variance(scores_lgbm)) # 0.0026262712086987383

#### Gradient Boosting Classifier: 

#### Random Forest Classifier:

* On choisit le LightGBM classifier principalement pour son fit time comparé aux autres.

### Optimisation des hyperparamètres du modèle le LightGBM classifier:

In [14]:
# Optimisation avec hyperopt
from hyperopt import fmin, tpe, hp, anneal, Trials
from random import *
from sklearn.metrics import fbeta_score, make_scorer

    
def lgbm_cv(params):
    
    params = {'n_estimators': params['n_estimators'], 
              'learning_rate': params['learning_rate'], 
              'num_leaves': params['num_leaves'],
              'colsample_bytree': params['colsample_bytree'],
              'subsample': params['subsample'],
              'min_child_weight': params['min_child_weight']
                }
    
      #'reg_alpha': params['reg_alpha'],
            #  'reg_lambda': params['reg_lambda'],
            
    # we use this params to create a new LGBM Regressor
    model = LGBMClassifier(**params)
    
    # and then conduct the cross validation with the same folds as before
    
    score = -cross_val_score(model, X_train_s,y_train_s, cv=4, scoring= 'roc_auc' , n_jobs=-1).mean()

    return score

In [ ]:
%%time

from random import *

# possible values of parameters
space={'n_estimators': hp.choice('n_estimators', [1e3,1e4,1e5]), 
        'learning_rate': hp.choice('learning_rate', [1e-3,1e-2,1e-1,1]),
        'num_leaves': hp.choice('num_leaves', [5,15,25,35,45,60,75]),  
        'colsample_bytree': hp.choice('colsample_bytree', [0.1,0.4,0.7,0.9,1]), 
        'subsample': hp.choice('subsample', [0.1,0.4,0.7,0.9,1]),
        'min_child_weight' : hp.choice('min_child_weight', [1e-5,1e-4,1e-3,1e-2,1e-1])
      }

  #'reg_alpha': hp.uniform('reg_alpha', 0, 100),
 #'reg_lambda': hp.uniform('reg_lambda', 0, 100),

#trials will contain logging information
trials = Trials()

best=fmin(lgbm_cv, # function to optimize
          space=space, 
          algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
          max_evals= 50, # maximum number of iterations
          trials=trials, # logging
         )


 64%|██████▍   | 32/50 [19:58<11:34, 38.58s/trial, best loss=?]

In [ ]:
# computing the score on the test set
model = LGBMClassifier( n_estimators= best['n_estimators'], 
              learning_rate= best['learning_rate'], 
              num_leaves= int(best['num_leaves']),
              colsample_bytree= best['colsample_bytree'],
              subsample= best['subsample'],
              min_child_weight= best['min_child_weight']
                      )
             #reg_alpha=  best['reg_alpha'],
              #reg_lambda= best['reg_lambda'],

In [ ]:
with timer(" process fit for lgbm"): #  process fit for lgbm - done in 1175s (20min)
        lgbm.fit(X_train_s,y_train_s)

In [ ]:
pred_lgbm= lgbm.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Class 0', 'Class 1']
print(classification_report(y_test, pred_lgbm, target_names=target_names))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_lgbm)

In [ ]:
print(stop)

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

### Optimisation des hyperparamètres avec un scoring métier:

#### Application de hyperopt:

In [ ]:
from hyperopt import fmin, tpe, hp, anneal, Trials
from random import *
from sklearn.metrics import fbeta_score, make_scorer

    
def lgbm_cv(params):
    
    params = {'num_leaves': int(params['num_leaves']), 
              'min_child_weight': params['min_child_weight'], 
             'subsample': params['subsample'],
              'colsample_bytree': params['colsample_bytree'],
             'reg_alpha': params['reg_alpha'],
             'reg_lambda': params['reg_lambda']
             }
    
    # we use this params to create a new LGBM Regressor
    model = LGBMClassifier(**params)
    
    # and then conduct the cross validation with the same folds as before
    fbeta_scorer = make_scorer(fbeta_score, beta=3) # beta=3
    score = -cross_val_score(model, X_train_s,y_train_s, cv=4, scoring= fbeta_scorer, n_jobs=-1).mean()

    return score

In [ ]:
%%time

from random import *

# possible values of parameters
space={'num_leaves': int(hp.quniform('num_leaves', 10, 70, 1)),
       'min_child_weight' : hp.quniform('min_child_weight', 1e-5, 1e4, 1),
       'subsample' : hp.quniform('subsample', 0.5, 1, 1),
       'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 1),
       'reg_alpha' : hp.quniform('reg_alpha', 0, 100, 1),
       'reg_lambda' : hp.quniform('reg_lambda', 0, 100, 1),
      }

# trials will contain logging information
trials = Trials()

best=fmin(lgbm_cv, # function to optimize
          space=space, 
          algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
          max_evals= 50, # maximum number of iterations
          trials=trials, # logging
         )


# computing the score on the test set
model = LGBMClassifier( n_estimators=10000, num_leaves= best['num_leaves'], min_child_weight= best['min_child_weight'],
                       subsample= best['subsample'], colsample_bytree= best['colsample_bytree'],
                       reg_alpha= best['reg_alpha'], reg_lambda= best['reg_lambda'])

# Wall time: 1h 7min 38s
                       

In [ ]:
print("Best score {:.3f} params {}".format( lgbm_cv(best), best))

In [ ]:
#model = LGBMClassifier( n_estimators=10000, colsample_bytree= 1.0, min_child_weight= 9924, 
#        num_leaves= 39, reg_alpha= 5.0, reg_lambda= 86.0, subsample= 1.0)

In [ ]:
with timer(" process model fit"): # process model fit - done in 656s (11 min)
        model.fit(X_train_s,y_train_s)

In [ ]:
pred_lgbm= model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Class 0', ' Class 1']
print(classification_report(y_test, pred_lgbm, target_names=target_names))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_lgbm,labels= [0,1])

In [ ]:
print(stop)

### Features importance

df_importance.sort_values(by= 'Importance', ascending= False).head(20)

### Features importance with LIME

In [ ]:
import lime 
from lime import lime_tabular

#single instance from the test set. We will use it to make explanations with LIME 
#idx = random.randint(1, len(X_test))
test_1 = X_test.iloc[1] # y_test.iloc[1] == 0

lime_explainer = lime_tabular.LimeTabularExplainer( training_data=np.array(X_train_s), feature_names=X_train_s.columns,
                                                    class_names=['0', '1'], mode='classification')
                                                                        
lime_exp = lime_explainer.explain_instance(data_row=test_1, predict_fn=model.predict_proba)

lime_exp.show_in_notebook(show_table=True)

In [ ]:
import lime 
from lime import lime_tabular

#single instance from the test set. We will use it to make explanations with LIME 
test_2 = X_test.iloc[180] # y_test.iloc[180]== 1

lime_explainer = lime_tabular.LimeTabularExplainer( training_data=np.array(X_train_s), feature_names=X_train_s.columns,
                                                    class_names=['0', '1'], mode='classification')
                                                                        
lime_exp = lime_explainer.explain_instance(data_row=test_2, predict_fn=model.predict_proba)

lime_exp.show_in_notebook(show_table=True)

In [ ]:
lime_exp.predict_proba

#### Feature Importances using "as_pyplot_figure()"

In [ ]:
with plt.style.context("ggplot"):
    lime_exp.as_pyplot_figure()

#### Retrieve Features Importances as List using "as_list()": 

In [ ]:
lime_exp.as_list()

#### Retrieve Features Importances as Dictionary using "as_map()":

In [ ]:
lime_exp.as_map()

In [ ]:
# Retrieve Average Local and Global Prediction Value:
print("Explanation Local Prediction  : ", lime_exp.local_pred)
print("Explanation Global Prediction : ", lime_exp.predict_proba)

#### Visualize Features Importances for Wrong Predictions

In [ ]:
import random

preds = model.predict(X_test)

false_preds = np.argwhere((preds != y_test.to_numpy())).flatten()

idx  = random.choice(false_preds)

#print("Prediction : ", breast_cancer.target_names[model.predict(X_test[idx].reshape(1,-1))[0]])
#print("Actual :     ", breast_cancer.target_names[Y_test[idx]])

explanation = lime_explainer.explain_instance(X_test.iloc[idx], model.predict_proba)

explanation.show_in_notebook()

* Piste d'amélioration du modèle: écarter les features avec forte contribution dans les FP:

(organization_type_legal_services), (organization_type_industry_type_3)

### SHAP:

In [ ]:
import shap
shap.initjs()

In [ ]:
# Here we use the Tree SHAP implementation integrated into Light GBM to explain the entire dataset:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

In [ ]:
# Visualize a single prediction:
shap.force_plot(explainer.expected_value[1], shap_values[1][0,:], X_test.iloc[0,:])

In [ ]:
# Visualize many predictions: we only visualize 100 individuals.
#  Impact of the individual feature on all 100 predictions. 
#In this chart, y-axis values represent predicted values for each sample and the x-axis represents 10 samples from 0-9.
shap.force_plot(explainer.expected_value[1], shap_values[1][:100,:], X_test.iloc[:100,:])

* SHAP Summary Plot: 

In [ ]:
# SHAP Summary Plot: 
# we use a density scatter plot of SHAP values for each feature to identify how much impact 
#each feature has on the model output for individuals in the validation dataset.
shap.summary_plot(shap_values, X_test)

### Serialisatin du modèle avec Joblib

In [ ]:
# import the library
import joblib
model_lgbm= model
# set the output object name
filename = 'model.joblib'
# serialize (save) the object, in this case "model"
joblib.dump(model_lgbm, filename)

### Création de l'API avec MLflow

In [ ]:
from mlflow.models.signature import infer_signature
signature = infer_signature(X_train_s, y_train_s)

In [ ]:
import mlflow
import mlflow.sklearn
mlflow.lightgbm.save_model(model, 'lgbm_model', signature=signature)

* Déploiement d'une API REST depuis le terminal:
 mlflow models serve -m C:/Users/marat/Downloads/Py-DS-ML-Bootcamp-master/OCR/P7/lgbm_model/ 

* Requête curl pour envoyer une requête:
** curl http://127.0.0.1:5000/invocations -H 'Content-Type: application/json' -d '{"data": [[x, y, z,etc]]}'

In [ ]:
* Avec la console, depuis le répertoire racine, lancer Streamlit avec la commande:
streamlit run C:/Users/marat/Downloads/Py-DS-ML-Bootcamp-master/OCR/P7/dashboard.py